## Import Required Libraries

In [122]:
import json
import time
import os
import csv
import json
import numpy as np
import pandas as pd
import random
from itertools import combinations
import math
import sys
import pickle
import spotipy_api
import random

## Load information about Playlist and Tracks

In [2]:
t0 = time.time()
# Load data matching playlist ID to Track ID
df_play_track = pd.read_csv('./data/track_w_playlist_id.csv')

# Load file containing the Track names
fr = open('./data/tracks_w_names.csv', 'r', encoding="utf-8")
reader = csv.reader(fr)
track_name = []
for row in reader:
    track_name.append(row[1])
fr.close()

In [3]:
# Load file containing candidate pair 
fr = open("./data/candidate_pair_1rows_10bands.pickle", "rb")
can_pairs = pickle.load(fr, encoding='bytes')
fr.close()

# Sort data to make it easier to search
can_pairs = can_pairs[can_pairs[:, 0].argsort()]

## Define Function to Recommend Tracks

In [4]:
def song_recommendation(playlist_track_ids, k, candidate_pairs, track_with_name):
    # Get all similar tracks using candidate pairs
    track_ids = np.array([], dtype=int)
    for i in playlist_track_ids:
        idx_start = np.searchsorted(candidate_pairs[:, 0], i, side='left')
        idx_end = np.searchsorted(candidate_pairs[:, 0], i, side='right')
        can_track_ids = candidate_pairs[np.array(range(idx_start,idx_end+1)),1]
        track_ids = np.append(track_ids, can_track_ids)
    
    # Get count of number of each recommended track
    track_rec_id_count = dict([])
    for track in track_ids:
        count = track_rec_id_count.get(track, 0)
        track_rec_id_count[track] = count + 1
    
    # Convert track dictionary to list for easier use
    track_vs_count = []
    for key, value in track_rec_id_count.items():
        track_vs_count.append([value, key])
    
    # Sort track recommendation by suggestion count
    track_vs_count.sort(reverse=True)
    track_vs_count = np.array(track_vs_count)
    
    # Return top K results
    final_ids = track_vs_count[0:k,1]
    track_names = []
    for ele in final_ids:
        track_names.append(track_with_name[ele])
    return track_names

## Playlist Addition Recommedation 

In [5]:
# Get list of tracks from a specific playlist
playlist_track_list = df_play_track[df_play_track['Playlist ID'] == 0]["Track ID"].to_list()

# Get number of input and test songs from playlist
num_input_songs = round(len(playlist_track_list)*0.8)
num_test_songs = len(playlist_track_list)-num_input_songs

# Get random set of input songs
random.shuffle(playlist_track_list)
input_songs = []
for i in range(0,num_input_songs):
    input_songs.append(playlist_track_list.pop())
    
# Get random set of test songs
test_songs = [] 
for i in range(0,num_test_songs):
    test_songs.append(playlist_track_list.pop())

# Get song recommendations
recom_songs = song_recommendation(input_songs, num_test_songs, can_pairs, track_name)

In [6]:
# Display recommendation songs
print('Recommendations:')
for i in range(0,num_test_songs):
    print(recom_songs[i])
    
# Display test songs
print('\nActual Songs:')
for i in range(0,num_test_songs):
    print(track_name[test_songs[i]])

Recommendations:
Sk8er Boi - Avril Lavigne
Promiscuous - Nelly Furtado
Eenie Meenie - Justin Bieber
Beautiful Girls - Sean Kingston
Fergalicious - Fergie
My First Kiss - feat. Ke$ha - 3OH!3
Right Now (Na Na Na) - Akon
Apologize - OneRepublic
Fire Burning - Sean Kingston
Hot In Herre - Nelly

Actual Songs:
Sk8er Boi - Avril Lavigne
Me & U - Cassie
Ice Box - Omarion
Closer - Ne-Yo
Year 3000 - Jonas Brothers
Livin on Sunday - Cris Cab
Somebody To Love - Justin Bieber
Promiscuous - Nelly Furtado
Yeah! - Usher
Replay - Iyaz


In [14]:
# Spotipy API
sp_api = spotipy_api.EasySpotipy()

Random playlist:
["'Round Midnight - Eldar Djangirov"
 "They Can't Take That Away From Me - Fred Astaire"
 'Mule - Remastered - Kenny Burrell'
 'I Hear A Rhapsody - Bill Evans, Jim Hall' 'Manhattan - Blossom Dearie'
 'More Than You Know - Blossom Dearie'
 'Lover Man (Oh Where Can You Be) - Blossom Dearie'
 "I'm A Fool To Want You - 2006 Digital Remaster - Donald Byrd"
 'Samatha - Matthew Halsall' 'What Stays - Till Brönner'
 'Dear Kathy - Benny Golson'
 "Gary's Theme - 2003 Remastered Version - Bill Evans"
 'You Go To My Head - Art Pepper'
 'The Peacocks - Anthony Wonsey Trio, Richie Goods, Tony Reedus'
 'When October Goes - John Brown'
 'Where We Used To Live - Esbjörn Svensson Trio'
 'Journey Into Melody - 2007 Digital Remaster/Rudy Van Gelder Edition - Stanley Turrentine'
 'I Still Believe in You - Eric Reed' 'The Nearness Of You - Bill Charlap'
 'Good Morning.Goodbye - Dan Cray Trio']


In [118]:
PLAYLIST_SIZE=30
INPUT_SIZE=10
NUM_SONGS_TO_RECOMMEND=20
TOTAL_SONGS_TO_PRINT=max(INPUT_SIZE + NUM_SONGS_TO_RECOMMEND, PLAYLIST_SIZE)

random_playlist = sp_api.getRandomPlaylist(min_tracks=PLAYLIST_SIZE, max_tracks=PLAYLIST_SIZE)
random_playlist_indices = random_playlist[:,1].tolist()
print(f'Random playlist:\n{random_playlist[:,2]}')

Random playlist:
['7 - Catfish and the Bottlemen' 'Shuffle - Bombay Bicycle Club'
 'Talk Your Mind - Riscas' 'Dancing On Glass - St. Lucia'
 'Breezeblocks - alt-J' 'Tongue Tied - Grouplove'
 'She Moves In Her Own Way - The Kooks' 'Flashlight - The Front Bottoms'
 'Holding On - The War On Drugs' 'Helena Beat - Foster The People'
 'Genghis Khan - Miike Snow' 'The Wolf - Mumford & Sons'
 'Fire - Diskopunk' 'Temple - Kings of Leon' 'Colours - Grouplove'
 'The Ghosts of Beverly Drive - Death Cab for Cutie'
 'Dance, Baby! - boy pablo' 'Fever - The Black Keys'
 'Our Own House - MisterWives' 'Girls - The 1975' 'Midnight City - M83'
 'Sex on Fire - Kings of Leon' 'Disparate Youth - Santigold'
 'Call It What You Want - Foster The People' 'In The Morning - Razorlight'
 'Chelsea Dagger - The Fratellis' 'Pumpin Blood - NONONO'
 'Notion - The Rare Occasions' 'XOXO - Cherub' 'Young Boy - snny']


In [116]:
# Randomize songs within playlist
random.shuffle(random_playlist_indices)

recom_songs = song_recommendation(random_playlist_indices[0:INPUT_SIZE], NUM_SONGS_TO_RECOMMEND, can_pairs, track_name)

In [117]:
random_playlist_names = []
for index in random_playlist_indices:
    random_playlist_names.append(track_name[index])

print('{:<65}{:^10}{:<65}'.format('Original', '----', 'Recommended'))
print('--------------------------------------------------------------------------------------')
for i in range(TOTAL_SONGS_TO_PRINT):
    if i <= INPUT_SIZE:
        print('{:<65}{:^10}{:<65}'.format(track_name[random_playlist_indices[i]], '----', track_name[random_playlist_indices[i]]))
    elif i > INPUT_SIZE and i < PLAYLIST_SIZE:
        print('{:<65}{:^10}{:<65}'.format(track_name[random_playlist_indices[i]], '----', recom_songs[i - INPUT_SIZE]))
    elif i > PLAYLIST_SIZE:
        print('{:<65}{:^10}{:<65}'.format('', '----', recom_songs[i - INPUT_SIZE]))
    if i == INPUT_SIZE:
        print(f'Inputs above -----------------------------------------------------------')

print('Songs in both: ')
print(list(set(recom_songs).intersection(random_playlist_names)))

Original                                                            ----   Recommended                                                      
--------------------------------------------------------------------------------------
It Was A Good Day - Ice Cube                                        ----   It Was A Good Day - Ice Cube                                     
Regulate - Warren G                                                 ----   Regulate - Warren G                                              
Check Yo Self - Remix; - Ice Cube                                   ----   Check Yo Self - Remix; - Ice Cube                                
Passin' Me By - The Pharcyde                                        ----   Passin' Me By - The Pharcyde                                     
Keep Their Heads Ringin' - LP Version; - Dr. Dre                    ----   Keep Their Heads Ringin' - LP Version; - Dr. Dre                 
Tonite - DJ Quik                                                   

In [121]:
def test_recommender(spotipy, track_name_list, candidate_pairs, playlist_size=30, input_size=10, num_songs_to_recommend=20):
    TOTAL_SONGS_TO_PRINT=max(input_size + num_songs_to_recommend, playlist_size)

    random_playlist = spotipy.getRandomPlaylist(min_tracks=playlist_size, max_tracks=playlist_size)
    random_playlist_indices = random_playlist[:,1].tolist()

    # Randomize songs within playlist
    random.shuffle(random_playlist_indices)

    # Get recommendations
    recom_songs = song_recommendation(random_playlist_indices[0:input_size], num_songs_to_recommend, candidate_pairs, track_name_list)

    random_playlist_names = []
    for index in random_playlist_indices:
        random_playlist_names.append(track_name_list[index])

    print('{:<65}{:^10}{:<65}'.format('Original', '----', 'Recommended'))
    print('--------------------------------------------------------------------------------------')
    for i in range(TOTAL_SONGS_TO_PRINT):
        if i <= input_size:
            print('{:<65}{:^10}{:<65}'.format(track_name_list[random_playlist_indices[i]], '----', track_name_list[random_playlist_indices[i]]))
        elif i > input_size and i < playlist_size:
            print('{:<65}{:^10}{:<65}'.format(track_name_list[random_playlist_indices[i]], '----', recom_songs[i - input_size]))
        elif i > playlist_size:
            print('{:<65}{:^10}{:<65}'.format('', '----', recom_songs[i - input_size]))
        if i == input_size:
            print(f'Inputs above -----------------------------------------------------------')

    print('Songs in both: ')
    print(list(set(recom_songs).intersection(random_playlist_names)))

In [124]:
for i in range(10):
    test_recommender(sp_api, track_name, can_pairs)

Original                                                            ----   Recommended                                                      
--------------------------------------------------------------------------------------
Heard It All Before - Sunshine Anderson                             ----   Heard It All Before - Sunshine Anderson                          
Right My Wrongs - Bryson Tiller                                     ----   Right My Wrongs - Bryson Tiller                                  
No Diggity - Blackstreet                                            ----   No Diggity - Blackstreet                                         
Let's Get Married - ReMarqable Remix - Jagged Edge                  ----   Let's Get Married - ReMarqable Remix - Jagged Edge               
I Can Love You - Mary J. Blige                                      ----   I Can Love You - Mary J. Blige                                   
Confessions Part II - Confessions Special Edition Version - Usher  